In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

In [23]:
df = pd.read_csv("../data/tourist_attractions.csv")
df.head(3)

,id,name,type,region,locality,lat,lon
0,1,"The ""second"" home shopping store ""IJ Churin an...",architecture,Amur region,Blagoveshchensk,127.548877,49.811568
1,2,"""Town of security officers""",architecture,Sverdlovsk region,Ekaterinburg,60.621271,56.841798
2,3,"""Palace for the beloved""",architecture,Kursk region,Safonovka,60.621271,51.491293


In [224]:
columnsTypeSchema = {
    "numeric": ["lat", "lon"],
    "category": ["type", "region", "locality"],
    "text": ["name"]
}

def cleanData(dataFrame, typeSchema):
    print("Очитска колонок и вставка дефолтных значений")
    print(f"cleanData - Строк: {dataFrame.shape[0]}; Колонок: {dataFrame.shape[1]};")
    print(f"cleanData - Колонки: {list(dataFrame.columns)}")
    print(f"cleanData - Типы колонок: {typeSchema}")
    # ЧИСЛА - Очищаю, заменяю значения и превращаю в числовые значения
    for column in typeSchema["numeric"]:
        df[column] = df[column].fillna(0).replace("Not found", 0)
        df[column] = df[column].astype('float64')
        
    # КАТЕГОРИИ - Пустые значения получают категорию [undefined]
    for column in typeSchema["category"]:
        df[column] = df[column].fillna('undefined')
        
    # Тескты - заменяю на пустую строку
    for column in typeSchema["text"]:
        df[column] = df[column].fillna('')
        
    return df

def transformScaleData(dataFrame, typeSchema):
    print("Нормализация - векторизация колонок")
    # Удаляю колонку с ИД
    dataFrame.drop("id", axis=1, inplace=True)
    
    # метки для категориальных колонок
    le = LabelEncoder()
    for column in typeSchema["category"]:
        dataFrame[column] = le.fit_transform(dataFrame[column])
    
    # нормализация для числовых колонок
    sc = StandardScaler()
    dataFrame[typeSchema["numeric"]] = sc.fit_transform(dataFrame[typeSchema["numeric"]])
    
    # Векторизация текстовых данных
    framesList = []
    for column in typeSchema["text"]:
        column_vec = TfidfVectorizer(min_df=2, ngram_range=(1, 3), stop_words=[])
        vecFrameData = column_vec.fit_transform(dataFrame[column])
        framesList.append(vecFrameData)
        # удаляю колонку из общего фрейма данных
        dataFrame.drop(column, axis=1, inplace=True)

    # Добавляю осташиеся колонки к списку текстовых фреймов
    framesList.append(dataFrame)
    
    # Объединяю массив фреймов в стек для поиска ближайших похожих
    trainStack = hstack(framesList)
    
    # вычисляю ближайшие похожие элементы
    nearestNeighbors = cosine_similarity(trainStack, trainStack)
    return nearestNeighbors
    
cleanedDataFrame = cleanData(df, columnsTypeSchema)
originalDataFrame = cleanedDataFrame.copy()

nearestNeighborsData = transformScaleData(cleanedDataFrame, columnsTypeSchema)
print(nearestNeighborsData)
originalDataFrame.head(5)

Очитска колонок и вставка дефолтных значений
cleanData - Строк: 5241; Колонок: 7;
cleanData - Колонки: ['id', 'name', 'type', 'region', 'locality', 'lat', 'lon']
cleanData - Типы колонок: {'numeric': ['lat', 'lon'], 'category': ['type', 'region', 'locality'], 'text': ['name']}
Нормализация - векторизация колонок
[[1.         0.95013753 0.98196235 ... 0.98017748 0.98074511 0.97976598]
 [0.95013753 1.         0.95866455 ... 0.95020364 0.965856   0.96176505]
 [0.98196235 0.95866455 1.         ... 0.99959217 0.99960475 0.9998501 ]
 ...
 [0.98017748 0.95020364 0.99959217 ... 1.         0.99846849 0.99915606]
 [0.98074511 0.965856   0.99960475 ... 0.99846849 1.         0.99987497]
 [0.97976598 0.96176505 0.9998501  ... 0.99915606 0.99987497 1.        ]]


,id,name,type,region,locality,lat,lon
0,1,"The ""second"" home shopping store ""IJ Churin an...",architecture,Amur region,Blagoveshchensk,127.548877,49.811568
1,2,"""Town of security officers""",architecture,Sverdlovsk region,Ekaterinburg,60.621271,56.841798
2,3,"""Palace for the beloved""",architecture,Kursk region,Safonovka,60.621271,51.491293
3,4,"""The House with The Firebird"" (manor Zhelyabov...",architecture,Tomsk region,Tomsk,85.050946,56.469513
4,5,"""House with the ghosts""",architecture,Novosibirsk region,Novosibirsk,82.958761,55.041787


In [225]:
results = {}

for idx, row in originalDataFrame.iterrows():
    # Вытянуть ИД соседних элементов и отсортировать по возрастанию
    nearestNeighborsIndices = nearestNeighborsData[idx].argsort()[:-100:-1]
    # Сопоставить ИД элементов с результатом значения близости соседства
    nearestNeighborsItems = [(nearestNeighborsData[idx][i], originalDataFrame['id'][i]) for i in nearestNeighborsIndices]
    # Записать результат для будущих сопоставлений
    results[row['id']] = nearestNeighborsItems[1:]

print("Готово!")

Готово!


In [226]:
# Получение оригинального объекта по ИД
def item(id):
    return originalDataFrame.loc[originalDataFrame['id'] == id]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Запрос " + str(num) + " рекомендаций для элемента:")
    print(item(item_id))
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        itemName = item(rec[1])["name"].tolist()[0].split(' - ')[0]
        print("Рекоммендация: ID=" + str(rec[1]) + "; " + itemName + " (Близость:" + str(rec[0]) + ")")

recommend(item_id=32, num=5)

Запрос 5 рекомендаций для элемента:
    id                                     name          type  \
31  32  The ensemble of the first girls' school  architecture   

               region      locality        lat        lon  
31  Sverdlovsk region  Ekaterinburg  60.592336  56.886105  
-------
Рекоммендация: ID=785; The building of the American Hotel (Близость:0.9999763483640955)
Рекоммендация: ID=845; The building is the gymnasium (high school №9) (Близость:0.9999760325299012)
Рекоммендация: ID=740; The building of the Palace of Youth (Близость:0.9999758138282879)
Рекоммендация: ID=950; The complex of the former hospital of Upper Iset plant (Близость:0.9999757505814544)
Рекоммендация: ID=688; Residential complex "The second house of the City Council" (Близость:0.999975450094366)
